In [1]:
import torch
from torch import nn
from src.utils.preprocess_utils import notes_to_midi, midi_to_note_indices, one_hot_encode
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [2]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, input_size]
        out, _ = self.lstm(x)  # out: [batch_size, sequence_length, hidden_size]
        out = self.fc(out)
        return out

In [3]:
# Load model
model_name = "2025-01-05_19-47-45_h128_l5_sl60"
model = torch.load(f'weights/model_1/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_10184\462686548.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_1/{model_name}')


LSTMModel(
  (lstm): LSTM(85, 128, num_layers=5, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [4]:
# Load input
input = midi_to_note_indices('data/input.mid', 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [5]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_sequence_one_hot = one_hot_encode(start_sequence, num_notes=num_notes)
    start_tensor = torch.tensor(start_sequence_one_hot, dtype=torch.float32).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(one_hot_encode(generated_sequence_tensor, num_notes=num_notes), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 6, 17, 33, 45, 50, 48, 19, 46, 44, 38, 34, 52, 49, 55, 57, 54, 66, 28, 30, 18, 57, 60, 62, 50, 60, 48, 53, 33, 63, 61, 57, 56, 53, 55, 52, 50, 51, 48, 54, 49, 52, 53, 51, 49, 50, 49, 50, 49, 47, 45, 43, 42, 40, 44, 42, 38, 40, 35, 19, 34, 31, 12, 30, 28, 29, 28, 33, 39, 38, 35, 19, 7, 25, 34, 42, 39, 27, 30, 39, 32, 44, 40, 28, 42, 39, 35, 38, 31, 19, 46, 43, 42, 40, 43, 45, 42, 40, 44, 41, 46, 44, 40, 42, 25, 18, 24, 35, 27, 31, 19, 28, 32, 29, 12, 28, 30, 18, 39, 32, 25, 28, 43, 45, 44, 42, 29, 32, 28, 39, 40, 38, 42, 39, 38, 35, 28, 35, 29, 18, 6, 18, 30, 42, 45, 33, 49, 25, 47, 49, 52, 55, 57, 54, 57, 55, 52, 53, 57, 53, 62, 66, 63, 65, 62, 68, 62, 70, 62, 64, 62, 74, 81, 77, 70, 66, 59, 57, 69, 77, 66, 63, 67, 73, 79, 62, 65, 78, 80, 80, 78, 75, 68, 80, 70, 62, 68, 80, 73, 66, 67]


In [6]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_1/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_1/2025-01-05_19-55-41_t0.3_w200_M_2025-01-05_19-47-45_h128_l5_sl60.mid
